In [31]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIXED #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIXED update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIXED Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIXED Place the HTML image tag in the line below into the app.layout code according to your design
#FIXED Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        style={'display': 'flex', 'alignItems': 'center'},
        children=[
            html.A(
                href='https://www.snhu.edu',
                children=[
                    html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        style={'height': '120px', 'marginRight': '40px'}
                        )
                    ]
                ),
            html.Div(
                children=[
                    html.Center(html.B(html.H1('CS-340 Dashboard'))),
                    html.P('Cade Bray - UID: CadeBray89', style={'margin': '0 20px'})
                    ],
                style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center'}
                )
            ]
        ),
    html.Hr(),
    html.Div(
        children=[
            #FIXED Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
            html.Button('Water Rescue', id='water', value='Water Rescue',n_clicks=0, style={'margin': '0 10px'}),
            html.Button('Mountain or Wilderness Rescue', id='mtn-wild', n_clicks=0, style={'margin': '0 10px'}),
            html.Button('Disaster Rescue or Individual Tracking', id='dis-ind', n_clicks=0, style={'margin': '0 10px'}),
            html.Button('Reset', id='reset', n_clicks=0, style={'margin': '0 10px'})
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                        #FIXED: Set up the features for your interactive data table to make it user-friendly for your client
                        #If you completed the Module Six Assignment, you can copy in the code you created here 
                        # Page size is set to 20 rows. Felt Like a good view of data with limited scrolling.
                        page_size = 20,
                        # Made row selection single so map could load. 
                        row_selectable = "single",
                        # Don't allow editing of view
                        editable = False,
                        # Using native sorting and filtering
                        filter_action = "native",
                        sort_action = "native",
                        # Allow for multi sorting options. Client may want to utlize this for new views.
                        sort_mode = "multi",
                        # Make column so it cannot be selected. Our data is grouped by row not column
                        column_selectable = False,
                        # Make it so row cannot be deleted.
                        row_deletable = False,
                        # Do not preselect a column. Our data is grouped by row.
                        selected_columns = [],
                        # set the pre-selected row to the first one (0)
                        selected_rows = [0],
                        page_action = "native",
                        # Set the current page to 0 (the first)
                        page_current = 0,
                        # Style of the cells. Text is aligned for clarity and white space is normal
                        style_cell = {"textAlign": "center", "whiteSpace": "normal"},
                        # Add a pop of color, make headers bold, and add padding so sort button isn't so
                        # close.
                        style_header = {
                            "backgroundColor": "lightblue", 
                            "fontWeight": "bold", 
                            "padding": "10px"
                        }
                    ),
    html.Br(),
    html.Hr(),
    html.P('Cade Bray - UID: CadeBray89', style={'margin': '0 20px'}),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id', 'columns')],
              [Input('water', 'n_clicks_timestamp'),
              Input('mtn-wild', 'n_clicks_timestamp'),
              Input('dis-ind', 'n_clicks_timestamp'),
              Input('reset', 'n_clicks_timestamp')],
             prevent_initial_call=True)
def update_dashboard(water, mtn_wild, dis_ind, reset):
    # FIXED Add code to filter interactive data table with MongoDB queries
    timestamps = {
        'water-rescue': water or 0,
        'mtn-wild': mtn_wild or 0,
        'dis-ind': dis_ind or 0,
        'reset': reset or 0
    }
    
    filter_type = max(timestamps, key=timestamps.get)
    print('updating dashboard - ', filter_type)
    
    if filter_type == 'water-rescue':
        filter_query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
        }
    elif filter_type == 'mtn-wild':
        filter_query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26.0, '$lte': 156.0}
        }
    elif filter_type == 'dis-ind':
        filter_query = {
            'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20.0, '$lte': 300.0}
        }
    else:
        filter_query = {}
    
    filter_data = db.read(filter_query)
    df = pd.DataFrame.from_records(filter_data)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')       
    return (data, columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIXED ####
    # add code for chart of your choice (e.g. pie chart) #
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Animals')
    return [
        dcc.Graph(            
            figure = fig
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geolocation chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here 
    row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Pinged your deployment. You successfully connected to your MongoDB!
Dash app running on http://127.0.0.1:12276/
updating dashboard -  water-rescue
updating dashboard -  reset
updating dashboard -  water-rescue
updating dashboard -  mtn-wild
updating dashboard -  dis-ind
updating dashboard -  reset
updating dashboard -  water-rescue
updating dashboard -  mtn-wild
updating dashboard -  dis-ind
updating dashboard -  reset
updating dashboard -  reset
